# Evaluate Semantic Kernel AI (ChatCompletion) Agents in Azure AI Foundry

## Objective

This sample demonstrates how to evaluate Semantic Kernel AI ChatCompletionAgents in Azure AI Foundry. It provides a step-by-step guide to set up the environment, create an agent, and evaluate its performance.

## Time
You can expect to complete this sample in approximately 20 minutes.

## Prerequisites
### Packages
- `semantic-kernel` installed (`pip install semantic-kernel`)
- `azure-ai-evaluation` SDK installed *(pip install latest version to include the SK converter)*
- An Azure OpenAI resource with a deployment configured

### Environment Variables
- For AzureChatService: *(related to SK Agent generating conversation)*
  - `api_key` The API key to access your Azure OpenAI resource.
  - `chat_deployment_name` The name of the chat model deployment (e.g., gpt-4-32k) used by your agent.
  - `endpoint` The full endpoint URL of your Azure OpenAI resource (e.g., https://your-resource.openai.azure.com).
- For evaluating agents:
  - `AZURE_OPENAI_ENDPOINT` The base endpoint for Azure OpenAI (e.g., https://<resource>.openai.azure.com).
  - `AZURE_OPENAI_API_KEY` The API key for your Azure OpenAI resource.
  - `AZURE_OPENAI_API_VERSION` The API version to use (e.g., 2024-05-01-preview).
  - `MODEL_DEPLOYMENT_NAME` The model deployment name used by the agent or evaluation flow (e.g., gpt-4).
- For Azure AI Foundry (Bonus):
  - `AZURE_SUBSCRIPTION_ID` Your Azure subscription ID.
  - `PROJECT_NAME` The name of your Azure AI Foundry project.
  - `RESOURCE_GROUP_NAME` The name of the resource group containing your Azure AI project.

### Create a AzureChatCompletion service - [reference](https://learn.microsoft.com/en-us/semantic-kernel/concepts/ai-services/chat-completion/?tabs=csharp-AzureOpenAI%2Cpython-AzureOpenAI%2Cjava-AzureOpenAI&pivots=programming-language-python)

In [1]:
import dotenv
dotenv.load_dotenv()

True

In [3]:
import os

print(os.getenv("AZURE_OPENAI_API_VERSION"))

None


In [4]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# You can do the following if you have set the necessary environment variables or created a .env file
chat_completion_service = AzureChatCompletion(service_id="my-service-id")

### Create a ChatCompletionAgent - [reference](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-types/chat-completion-agent?pivots=programming-language-python)

In [5]:
from semantic_kernel.functions import kernel_function
from typing import Annotated

# This is a sample plugin that provides tools
class MenuPlugin:
    """A sample Menu Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of specials from the menu.")
    def get_specials(self) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    @kernel_function(description="Provides the price of the requested menu item.")
    def get_item_price(
        self, menu_item: Annotated[str, "The name of the menu item."]
    ) -> Annotated[str, "Returns the price of the menu item."]:
        return "$9.99"

In [6]:
from semantic_kernel.agents import ChatCompletionAgent

# Create the agent by directly providing the chat completion service
agent = ChatCompletionAgent(
    service=chat_completion_service,
    name="Chef",
    instructions="Answer questions about the menu.",
    plugins=[MenuPlugin()],
)

In [7]:
thread = None

user_inputs = [
    "Hello",
    "What is the special drink today?",
    "What does that cost?",
    "Thank you",
]

for user_input in user_inputs:
    response = await agent.get_response(messages=user_input, thread=thread)
    print(f"## User: {user_input}")
    print(f"## {response.name}: {response}\n")
    thread = response.thread

## User: Hello
## Chef: Hello! How can I assist you today? Do you have any questions about the menu?

## User: What is the special drink today?
## Chef: The special drink today is Chai Tea. Would you like to know more about it or anything else on the menu?

## User: What does that cost?
## Chef: The Chai Tea costs $9.99. Would you like to know about anything else?

## User: Thank you
## Chef: You're welcome! If you have any more questions or need assistance in the future, feel free to ask. Enjoy your day!



### Converter

In [9]:
from azure.ai.evaluation import SKAgentConverter

# Get the avaiable turn indices for the thread, 
# useful for selecting a specific turn for evaluation
turn_indices = await SKAgentConverter._get_thread_turn_indices(thread=thread)
print(f"Available turn indices: {turn_indices}")

Available turn indices: [0, 1, 2, 3]


In [10]:
converter = SKAgentConverter()

# Get a single agent run data
evaluation_data_single_run = await converter.convert(
    thread=thread,
    turn_index=2, # Specify the turn index you want to evaluate
    agent=agent # Pass it to include the instructions and plugins in the evaluation data
)

Class SKAgentConverter: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [12]:
evaluation_data_single_run.keys()

dict_keys(['query', 'response', 'tool_definitions'])

In [13]:
import json

file_name = "evaluation_data.jsonl"
# Save the agent thread data to a JSONL file (all turns)
evaluation_data = await converter.prepare_evaluation_data(threads=[thread], filename=file_name, agent=agent)
# print(json.dumps(evaluation_data, indent=4))
len(evaluation_data) # number of turns in the thread

4

### Setting up evaluator

We will select the following evaluators to assess the different aspects relevant for agent quality: 

- [Intent resolution](https://aka.ms/intentresolution-sample): measures the extent of which an agent identifies the correct intent from a user query. Scale: integer 1-5. Higher is better.
- [Tool call accuracy](https://aka.ms/toolcallaccuracy-sample): evaluates the agent’s ability to select the appropriate tools, and process correct parameters from previous steps. Scale: float 0-1. Higher is better.
- [Task adherence](https://aka.ms/taskadherence-sample): measures the extent of which an agent’s final response adheres to the task based on its system message and a user query. Scale: integer 1-5. Higher is better.


In [14]:
import os
from pprint import pprint

from azure.ai.evaluation import (
    ToolCallAccuracyEvaluator,
    AzureOpenAIModelConfiguration,
    IntentResolutionEvaluator,
    TaskAdherenceEvaluator,
)

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)

intent_resolution = IntentResolutionEvaluator(model_config=model_config)

tool_call_accuracy = ToolCallAccuracyEvaluator(model_config=model_config)

task_adherence = TaskAdherenceEvaluator(model_config=model_config)

Class IntentResolutionEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ToolCallAccuracyEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class TaskAdherenceEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [15]:
# Test a single evaluation run
evaluator = ToolCallAccuracyEvaluator(model_config=model_config)

# evaluation_data_single_run.keys() # query, response, tool_definitions
res = evaluator(**evaluation_data_single_run)
print(json.dumps(res, indent=4))

{
    "tool_call_accuracy": 1.0,
    "tool_call_accuracy_result": "pass",
    "tool_call_accuracy_threshold": 0.8,
    "per_tool_call_details": [
        {
            "tool_call_accurate": true,
            "tool_call_accurate_reason": "The input data should get a score of 1 because the tool call is directly relevant to the user's query about the cost of 'Chai Tea', the parameters match the tool definition, and the values are correctly inferred from the conversation.",
            "tool_call_id": "call_tacVY3lPvW6bHAQrpaIpW0pp"
        }
    ]
}


#### Bonus - run on perviously saved file for all turns

In [ ]:
from azure.ai.evaluation import evaluate

response = evaluate(
    data=file_name,
    evaluators={
        "tool_call_accuracy": tool_call_accuracy,
        "intent_resolution": intent_resolution,
        "task_adherence": task_adherence,
    },
    azure_ai_project={
        "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
        "project_name": os.environ["PROJECT_NAME"],
        "resource_group_name": os.environ["RESOURCE_GROUP_NAME"],
    },
)

pprint(f'AI Foundary URL: {response.get("studio_url")}')

## Inspect results on Azure AI Foundry

Go to AI Foundry URL for rich Azure AI Foundry data visualization to inspect the evaluation scores and reasoning to quickly identify bugs and issues of your agent to fix and improve.

In [ ]:
# alternatively, you can use the following to get the evaluation results in memory

# average scores across all runs
pprint(response["metrics"])